In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

#Load the model and Tokenizer
model_id = "google/gemma-2b-it"
#
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)


/Users/adityamakkar/Desktop/CS/overleafCopilot/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/adityamakkar/Desktop/CS/overleafCopilot/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
def generate_prompt(data_point):
    """Gen. input text based on a prompt, task instruction, (context info.), and answer

    :param data_point: dict: Data point
    :return: dict: tokenzed prompt
    """

    # Generate prompt
    prefix_text = 'Below is an instruction that describes a task. Write a response that ' \
               'appropriately completes the request.\n\n'
    # Samples with additional context into.
    if data_point['input']:
        text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} here are the inputs {data_point["input"]} <end_of_turn>\n<start_of_turn>model{data_point["output"]} <end_of_turn>"""
    # Without
    else:
        text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} <end_of_turn>\n<start_of_turn>model{data_point["output"]} <end_of_turn>"""
    return text


Dataset({
    features: ['instruction', 'output', 'input', 'prompt'],
    num_rows: 3702
})

In [8]:
from peft import LoraConfig, PeftModel, get_peft_model
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="mps")
model.gradient_checkpointing_enable()
#
print(model)

def find_all_linear_names(model):
  cls = torch.nn.Linear
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)
#
modules = find_all_linear_names(model)

limit=4

lora_config = LoraConfig(
    r=4,
    lora_alpha=2,
    target_modules=modules if len(modules) < limit else modules[:limit],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

/Users/adityamakkar/Desktop/CS/overleafCopilot/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.95s/it]


GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaRM

/Users/adityamakkar/Desktop/CS/overleafCopilot/.venv/lib/python3.12/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [10]:
import transformers

from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side='right'


trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    max_seq_length=2500,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=0.03,
        max_steps=100,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="adamw_torch",
        save_strategy="epoch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
#
trainer.train()

Map: 100%|██████████| 371/371 [00:00<00:00, 7403.70 examples/s]
/Users/adityamakkar/Desktop/CS/overleafCopilot/.venv/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
  1%|          | 1/100 [00:12<20:03, 12.15s/it]

{'loss': 3.3987, 'grad_norm': 0.43529394268989563, 'learning_rate': 0.00019805941782534764, 'epoch': 0.0}


  2%|▏         | 2/100 [00:22<17:53, 10.96s/it]

{'loss': 3.4251, 'grad_norm': 0.4118957221508026, 'learning_rate': 0.00019605881764529358, 'epoch': 0.0}


  3%|▎         | 3/100 [00:32<17:10, 10.62s/it]

...

 25%|██▌       | 25/100 [05:48<28:59, 23.19s/it]

{'loss': 2.187, 'grad_norm': 0.7943320274353027, 'learning_rate': 0.00015004501350405122, 'epoch': 0.03}


 26%|██▌       | 26/100 [06:06<27:02, 21.92s/it]

{'loss': 1.9732, 'grad_norm': 0.8320187330245972, 'learning_rate': 0.00014804441332399721, 'epoch': 0.03}


KeyboardInterrupt: 